In [1]:
#load source file into RDD, as line objects
lines = sc.textFile("/SparkCourse/1800.csv")

In [2]:
print (lines.top(5))

['ITE00100554,18001231,TMIN,25,,,E,', 'ITE00100554,18001231,TMAX,50,,,E,', 'ITE00100554,18001230,TMIN,31,,,E,', 'ITE00100554,18001230,TMAX,50,,,E,', 'ITE00100554,18001229,TMIN,16,,,E,']


In [57]:
#function to split on comma, and return as [((station, TempDateMonth), (tempType, Temp))]
#return only YearMonth value in TempDateMonth 
def splitOnComma(line):
    line = line.split(',')
    return ((line[0], line[1][:6]), (line[2], int(line[3])))

##### Using mapValues which doesnot reshuffle the partitions again, so planned the func 'splitOnComma' to return (key,values) as needed
##### Note: The value part of the key, value pair will only include the first element in the values if not grouped by brackets

      ex1: [(('ITE00100554', '18000101'), 'TMIN', -148)  #here value is 'TMIN', value[1] returns 'M'
      ex2: [(('ITE00100554', '18000101'), ('TMIN', -148))  #here value is ('TMIN', -148), value[1] returns -148

In [58]:
#Transform to (key, value) RDD1 as [((station, minTempDate), minTemp)], RDD2 as [((station, maxTempDate), maxTemp)], then filter on approptiate tempType
#using mapValues which doesnot reshuffle the partitions again, so planned the func 'splitOnComma' to return keyvalues as needed

rddLinesMin = lines.map(lambda line: splitOnComma(line)).filter(lambda line: line[1][0] == 'TMIN').mapValues(lambda x: x[1])
rddLinesMax = lines.map(lambda line: splitOnComma(line)).filter(lambda line: line[1][0] == 'TMAX').mapValues(lambda x: x[1])

In [59]:
print ('rddLinesMin: ',rddLinesMin.take(5))
print ('\nrddLinesMax: ',rddLinesMax.take(5))

rddLinesMin:  [(('ITE00100554', '180001'), -148), (('EZE00100082', '180001'), -135), (('ITE00100554', '180001'), -125), (('EZE00100082', '180001'), -130), (('ITE00100554', '180001'), -46)]

rddLinesMax:  [(('ITE00100554', '180001'), -75), (('EZE00100082', '180001'), -86), (('ITE00100554', '180001'), -60), (('EZE00100082', '180001'), -44), (('ITE00100554', '180001'), -23)]


In [60]:
#Transform to calculate the minimum temperature and maximum temperature records for each key(station, Tempdate)
rddStationMinTemp = rddLinesMin.reduceByKey(lambda x, y: x if x <= y else y)
rddStationMaxTemp = rddLinesMax.reduceByKey(lambda x, y: x if x >= y else y)

In [63]:
print ('rddStationMinTemp: ',rddStationMinTemp.take(5))
print ('\rddStationMaxTemp: ',rddStationMaxTemp.take(5))

rddStationMinTemp:  [(('ITE00100554', '180007'), 163), (('ITE00100554', '180009'), 100), (('EZE00100082', '180001'), -135), (('ITE00100554', '180001'), -148), (('EZE00100082', '180006'), 82)]
ddStationMaxTemp:  [(('ITE00100554', '180007'), 319), (('ITE00100554', '180009'), 250), (('EZE00100082', '180001'), 69), (('ITE00100554', '180001'), 98), (('EZE00100082', '180006'), 250)]


In [64]:
# Outerjoin the RDD1 and RDD2 on Keys, by Transforming into [((station, maxTempDate), (minTemp, maxTemp)]
joinRDD1RDD2 = rddStationMinTemp.fullOuterJoin(rddStationMaxTemp).sortBy(lambda x: x[0][1])

In [65]:
# Action to collect RDD into python object : to Return all the elements of the dataset RDD as an array at the driver program
result = joinRDD1RDD2.collect()
print ('stationID, dateMonth, minTemp, MaxTemp')
for i, j in result:
    print (i[0], i[1], j[0], j[1])

stationID, dateMonth, minTemp, MaxTemp
ITE00100554 180001 -148 98
EZE00100082 180001 -135 69
EZE00100082 180002 -102 67
ITE00100554 180002 0 129
ITE00100554 180003 0 141
EZE00100082 180003 -130 123
ITE00100554 180004 75 235
EZE00100082 180004 50 286
ITE00100554 180005 96 269
EZE00100082 180005 100 282
EZE00100082 180006 82 250
ITE00100554 180006 138 288
ITE00100554 180007 163 319
EZE00100082 180007 120 306
EZE00100082 180008 117 323
ITE00100554 180008 143 323
ITE00100554 180009 100 250
EZE00100082 180009 85 230
ITE00100554 180010 31 200
EZE00100082 180010 -11 174
EZE00100082 180011 -6 173
ITE00100554 180011 19 150
EZE00100082 180012 -83 96
ITE00100554 180012 -23 91


In [66]:
###################################################################################
### find Max and Min Temperatures recorded per Station and Month date
###################################################################################

#broiler plate stuff to be un commented
####from pyspark import SparkConf, SparkContext

####conf = SparkConf().SetMaster("local").setAppName("MovieCountByRatings")
####sc = SparkContext(conf = conf)

#load source file into RDD, as line objects
lines = sc.textFile("/SparkCourse/1800.csv")

#function to split on comma, and return as [((station, TempDateMonth), (tempType, Temp))]
#return only YearMonth value in TempDateMonth 
def splitOnComma(line):
    line = line.split(',')
    return ((line[0], line[1][:6]), (line[2], int(line[3])))

#Transform to (key, value) RDD1 as [((station, minTempDate), minTemp)], RDD2 as [((station, maxTempDate), maxTemp)], then filter on approptiate tempType
#using mapValues which doesnot reshuffle the partitions again, so planned the func 'splitOnComma' to return keyvalues as needed
rddLinesMin = lines.map(lambda line: splitOnComma(line)).filter(lambda line: line[1][0] == 'TMIN').mapValues(lambda x: x[1])
rddLinesMax = lines.map(lambda line: splitOnComma(line)).filter(lambda line: line[1][0] == 'TMAX').mapValues(lambda x: x[1])

#Transform to calculate the minimum temperature and maximum temperature records for each key(station, Tempdate)
rddStationMinTemp = rddLinesMin.reduceByKey(lambda x, y: x if x <= y else y)
rddStationMaxTemp = rddLinesMax.reduceByKey(lambda x, y: x if x >= y else y)

# Outerjoin the RDD1 and RDD2 on Keys, by Transforming into [((station, maxTempDate), (minTemp, maxTemp)]
joinRDD1RDD2 = rddStationMinTemp.fullOuterJoin(rddStationMaxTemp).sortBy(lambda x: x[0][1])

# Action to collect RDD into python object : to Return all the elements of the dataset RDD as an array at the driver program
result = joinRDD1RDD2.collect()
print ('stationID, dateMonth, minTemp, MaxTemp')
for i, j in result:
    print (i[0], i[1], j[0], j[1])


stationID, dateMonth, minTemp, MaxTemp
ITE00100554 180001 -148 98
EZE00100082 180001 -135 69
EZE00100082 180002 -102 67
ITE00100554 180002 0 129
ITE00100554 180003 0 141
EZE00100082 180003 -130 123
ITE00100554 180004 75 235
EZE00100082 180004 50 286
ITE00100554 180005 96 269
EZE00100082 180005 100 282
EZE00100082 180006 82 250
ITE00100554 180006 138 288
ITE00100554 180007 163 319
EZE00100082 180007 120 306
EZE00100082 180008 117 323
ITE00100554 180008 143 323
ITE00100554 180009 100 250
EZE00100082 180009 85 230
ITE00100554 180010 31 200
EZE00100082 180010 -11 174
EZE00100082 180011 -6 173
ITE00100554 180011 19 150
EZE00100082 180012 -83 96
ITE00100554 180012 -23 91
